In [ ]:
import duckdb
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
db = duckdb.connect(":memory:")
db

In [ ]:
db.execute("create or replace view process as select * from read_parquet('/data/acme3/stdview-20231105-20231120/process.parquet')")

In [ ]:
db.execute("create or replace view process_path as select * from read_parquet('/data/acme3/stdview-20231105-20231120/process_path.parquet')")

In [ ]:
db.execute("describe process").df()

In [ ]:
%%time
df = db.execute("select hostname, pid_hash, parent_pid_hash, process_name, args from process").df()
df

In [ ]:
%%time
wintaps = df.loc[df["process_name"] == "wintap.exe"]
wintaps

In [ ]:
wintaps["hostname"].value_counts()

In [ ]:
children0 = pd.merge(df, wintaps[["pid_hash"]], left_on="parent_pid_hash", right_on="pid_hash", how="inner", suffixes=("", "_discard"))
children0

In [ ]:
children0["process_name"].value_counts()

In [ ]:
children1 = pd.merge(df, children0[["pid_hash"]], left_on="parent_pid_hash", right_on="pid_hash", how="inner", suffixes=("", "_discard"))
children1

In [ ]:
children1.loc[children1["process_name"] == "winlogon.exe"]

In [ ]:
df.loc[df["pid_hash"] == "E68DD57B8393A2E74C988AA32614C16C"]

In [ ]:
db.sql("describe process_path").df()

In [ ]:
num_processes = np.sum(db.sql("select count(*) as num from process union select count(*) from process_path").df()["num"])
num_processes

In [ ]:
wintaps = db.sql(
    """
    select hostname, pid_hash, parent_pid_hash, process_name, ptree
    from process_path
    where ptree like '%wintap%'
    """
).df()
wintaps

In [ ]:
wintaps.shape[0] / num_processes

In [ ]:
amazon_ssm = db.sql(
    """
    select hostname, pid_hash, parent_pid_hash, process_name, ptree
    from process_path
    where ptree like '%amazon-ssm%'
    """
).df()
amazon_ssm

In [ ]:
interesting = db.sql(
    """
    select hostname, pid_hash, parent_pid_hash, process_name, ptree
    from process_path
    where ptree not like '%wintap%' and ptree not like '%amazon-ssm%'
    """
).df()
interesting

In [ ]:
process_cmdlines = db.sql(
    """
    select process.process_path, process.args
    from process
    inner join process_path using (pid_hash)
    where process_path.ptree not like '%wintap%' and process_path.ptree not like '%amazon-ssm%'
    """
).df().fillna("")
process_cmdlines

In [ ]:
cmdlines = (process_cmdlines["process_path"] + " " + process_cmdlines["args"]).str.strip()
cmdlines.value_counts().to_frame()